In [2]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
import sh
import pandas as pd
from datetime import datetime
datetime.now()
import matplotlib.pyplot as plt
print(sc.version)

2.4.0-cdh6.3.0


In [3]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [5]:
path='/user/ivy2/Tweets/'

In [4]:
# !hadoop fs -ls /user/ivy2/Tweets

In [6]:
tweets_all = spark.read.json(path).withColumn('coal_text',F.lower(F.coalesce(F.col('extended_tweet.full_text'), F.col('text')))) \
                                    .withColumn('uchicago', F.when((F.col('coal_text').contains('university of chicago')) |
                                                                   (F.col('coal_text').contains('universityofchicago')) |
                                                                   (F.col('coal_text').contains('uchicago')) |
                                                                   (F.col('coal_text').contains('uofc')), 'uchicago')) \
                                    .withColumn('cornell', F.when(F.col('coal_text').contains('cornell'), 'cornell')) \
                                    .withColumn('stanford', F.when(F.col('coal_text').contains('stanford'), 'stanford')) \
                                    .withColumn('cm', F.when((F.col('coal_text').contains('carnegiemellon')) |
                                                             (F.col('coal_text').contains('carnegie mellon')), 'cm')) \
                                    .withColumn('school', F.coalesce('uchicago', 'cornell', 'stanford', 'cm')) \
                                    .drop('uchicago') \
                                    .drop('cornell') \
                                    .drop('cm') \
                                    .drop('stanford')

In [7]:
tweets_uni = tweets_all.filter(~F.col('school').isNull())

In [7]:
tweets_uni.groupBy('school').count()

school,count
uchicago,800086
stanford,1794758
rutgers,321657
cornell,360399


In [8]:
tweets_uni = tweets_uni.filter(F.col('lang')=='en') \
                        .select([F.col('id').alias('tweet_id'),
                                 F.col('created_at').alias('tweet_dt'),
                                 F.col('source'),
                                 F.col('school'),
                                 F.col('in_reply_to_user_id').alias('is_reply'), 
                                 F.col('quoted_status'),
                                 F.col('retweeted_status'),
                                 F.col('entities.media.type').alias('media_type'),
                                 F.col('entities.user_mentions.id').alias('user_mentions_id'),
                                 F.col('favorite_count'),
                                 F.col('retweet_count'),
                                 F.col('user.created_at').alias('user_created_at'), 
                                 F.col('user.favourites_count').alias('user_favorites_count'),
                                 F.col('user.followers_count').alias('user_followers_count'),
                                 F.col('user.friends_count').alias('user_friends_count'),
                                 F.col('user.id').alias('user_id'),
                                 F.col('user.location').alias('user_location'),
                                 F.col('user.lang').alias('user_lang'), 
                                 F.col('user.verified').alias('user_verified')]).na.drop('all')
#                                  F.col('coal_text')

In [9]:
tweets_uni = tweets_uni.withColumn('is_quote', ~F.col('quoted_status').isNull()) \
                .withColumn('is_retweet', ~F.col('retweeted_status').isNull()) \
                .withColumn('is_reply', ~F.col('is_reply').isNull()) \
                .drop(F.col('quoted_status')).drop(F.col('retweeted_status')).cache()

In [10]:
tweets_uni = tweets_uni.withColumn('clean_text', F.regexp_replace(F.col('coal_text'),r'[^\w\s]',''))

tweets_uni = tweets_uni.withColumn("clean_text",F.regexp_replace(F.lower(F.col("clean_text")), r"[0-9]+", ""))

tweets_uni = tweets_uni.withColumn("clean_text",F.regexp_replace(F.col("clean_text"), r"http\S+", ""))

tweets_uni = tweets_uni.drop('coal_text')

In [10]:
tweets_uni = tweets_uni.withColumn('media', ~F.col('media_type').isNull()).drop('media_type')
tweets_uni = tweets_uni.withColumn('media', F.col('media').cast('integer'))
tweets_uni = tweets_uni.withColumn('user_verified', F.col('user_verified').cast('integer'))
tweets_uni = tweets_uni.withColumn('is_reply', F.col('is_reply').cast('integer'))
tweets_uni = tweets_uni.withColumn('is_quote', F.col('is_quote').cast('integer'))
tweets_uni = tweets_uni.withColumn('is_retweet', F.col('is_retweet').cast('integer'))
tweets_uni = tweets_uni.withColumn('user_mentions', ~F.col('user_mentions_id').isNull()).drop('user_mentions_id')
tweets_uni = tweets_uni.withColumn('user_mentions', F.col('user_mentions').cast('integer'))

In [12]:
# tweets_uni = tweets_uni.withColumn('is_reply', F.col('is_reply').cast('integer')) \
#         .withColumn('user_verified', F.col('user_verified').cast('integer')) \
#         .withColumn('is_quote', F.col('is_quote').cast('integer')) \
#         .withColumn('is_retweet', F.col('is_retweet').cast('integer')) \
#         .withColumn('media', F.col('media').cast('integer')) \
#         .withColumn('user_mentions', F.col('user_mentions').cast('integer'))

In [14]:
tweets_uni.printSchema()

root
 |-- tweet_id: long (nullable = true)
 |-- tweet_dt: string (nullable = true)
 |-- source: string (nullable = true)
 |-- school: string (nullable = true)
 |-- is_reply: integer (nullable = false)
 |-- favorite_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- user_created_at: string (nullable = true)
 |-- user_favorites_count: long (nullable = true)
 |-- user_followers_count: long (nullable = true)
 |-- user_friends_count: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- user_location: string (nullable = true)
 |-- user_lang: string (nullable = true)
 |-- user_verified: integer (nullable = true)
 |-- is_quote: integer (nullable = false)
 |-- is_retweet: integer (nullable = false)
 |-- media: integer (nullable = false)
 |-- user_mentions: integer (nullable = false)



### Extract variables for local analysis

In [17]:
tweets_uni.write.format('com.databricks.spark.csv').mode('overwrite').option('header','true').save('/user/jhuff1/project/tweet_variables')

In [24]:
!hadoop fs -getmerge '/user/jhuff1/project/tweet_variables' '/home/jhuff1/tweet_variables.csv'

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0


In [25]:
!hadoop fs -rm -r '/user/jhuff1/project/tweet_variables'

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
2021-03-11 23:34:50,501 INFO  [main] fs.TrashPolicyDefault (TrashPolicyDefault.java:moveToTrash(182)) - Moved: 'hdfs://nameservice1/user/jhuff1/project/tweet_variables' to trash at: hdfs://nameservice1/user/jhuff1/.Trash/Current/user/jhuff1/project/tweet_variables
